In [8]:
# import lancedb

# uri = "./data/sample-lancedb"
# db = lancedb.connect(uri)
# table = db.create_table("my_table",
#                          data=[{"vector": [3.1, 4.1], "item": "foo", "price": 10.0},
#                                {"vector": [5.9, 26.5], "item": "bar", "price": 20.0}])
# result = table.search([100, 100]).limit(2).to_pandas()

In [6]:
import pandas as pd

In [93]:
import datasets 
data = datasets.load_dataset("json",data_files = "./data/train_zh_0.json")

Extracting data files: 100%|████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 202.85it/s]


Dataset json downloaded and prepared to C:/Users/131655/.cache/huggingface/datasets/json/default-380481cb1dbc2bf8/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.63it/s]


In [102]:
data_df = data["train"].to_pandas()

In [103]:
data_df = data_df.head(20000)

In [24]:
import jieba
import lancedb
uri = "./data/sample-lancedb"
db = lancedb.connect(uri)

In [108]:
querys = data_df['instruction'].to_list()

In [109]:
ans = data_df['output'].to_list()

In [112]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-base-zh-v1.5')
embeddings_query = model.encode(querys, normalize_embeddings=True)

In [122]:
querys_seg = [" ".join(jieba.lcut(i)) for i in  querys]

In [124]:
datas =[]
for i in range(20000):
    data = {}
    data["query"] = querys[i]
    data["answer"] = ans[i]
    data["vector"] = list(embeddings_query[i])
    data["queryseg"] = querys_seg[i]
    datas.append(data)

In [125]:


table = db.create_table("medical",datas)

In [126]:
table.to_pandas().head(1)

,query,answer,vector,queryseg
0,血热的临床表现是什么?,初发或复发病不久。皮疹发展迅速，呈点滴状、钱币状或混合状。常见丘疹、斑丘疹、大小不等的斑片，...,"[-0.011770768, 0.04241949, -0.0045774174, -0.0...",血热 的 临床表现 是 什么 ?


In [127]:
table.create_fts_index("queryseg")

In [139]:
data_text = table.search("怀孕 高血压 是 什么 原因").limit(10).select(["queryseg"])

In [140]:
data_text.to_df()

C:\Users\131655\AppData\Local\Temp\ipykernel_18116\1791037122.py:1: DeprecatedWarning: to_df is deprecated as of 0.3.1 and will be removed in 0.4.0. Use to_pandas() instead
  data_text.to_df()


,queryseg,score
0,怀孕 宫颈 出血 是 什么 原因,12.341222
1,怀孕 十一 周有 积液 是 什么 原因 引起 的,11.322922
2,怀孕 了 会 恶心 什么 原因 ， 本人 下体 特别 痒 ， 而且 内衣 上 总是 黄黄的 ...,9.998762
3,高血压 会 导致 中风 吗 ， 最近 去 看 我 爷爷 ， 发觉 爷爷 的 嘴巴 歪 着 ，...,9.527532
4,怀孕 三个 月 先兆 流产 的 原因,9.524203
5,高血压 危象 的 临床表现 是 什么 ?,9.136556
6,高血压 肾病 和 肾性 高血压 有 什么 不同 ？ ， 伯父 上个月 高血压 ， 家里人 赶...,9.134981
7,高血压 体格检查 是 怎么样 ？ ， 这 几天 有人 建议 我 去 医院 做 高血压 体格检...,9.096700
8,妊娠期 高血压 疾病 的 并发症 是 什么 ？,8.870636
9,继发性 高血压 的 疾病 病因 是 什么 ?,8.870636


In [150]:
query_pred = model.encode("怀孕高血压疾病的推荐药有些什么？", normalize_embeddings=True)

In [151]:
data_embed =  table.search(query_pred).limit(10)

In [152]:
data_embed.to_df()

C:\Users\131655\AppData\Local\Temp\ipykernel_18116\496871881.py:1: DeprecatedWarning: to_df is deprecated as of 0.3.1 and will be removed in 0.4.0. Use to_pandas() instead
  data_embed.to_df()


,query,answer,vector,queryseg,_distance
0,妊娠期高血压疾病的推荐药有些什么？,川芎嗪联合硫酸镁；钙；低分子肝素钠；硫酸镁加丹参；还原型谷胱甘肽联合肝素；硫酸镁治疗；组织因...,"[-0.029837258, 0.008759381, -0.016733877, -0.0...",妊娠期 高血压 疾病 的 推荐 药 有些 什么 ？,0.087203
1,妊娠高血压的推荐药有些什么？,硫酸镁治疗；中药川芎嗪+硫酸镁；氯沙坦；联合用药；硫酸镁注射液；硫酸镁联合小剂量阿司匹林；硝...,"[-0.027922079, 0.010627506, -0.010434828, -0.0...",妊娠 高血压 的 推荐 药 有些 什么 ？,0.114372
2,中度妊娠期高血压的推荐药有些什么？,硝苯地平联合拉贝洛尔；拉贝洛尔；硝苯地平,"[-0.017963426, 0.007346955, -0.03789774, -0.00...",中度 妊娠期 高血压 的 推荐 药 有些 什么 ？,0.225096
3,高血压症的推荐药有些什么？,六味地黄丸；非洛地平；黄耆；钙通道阻滞剂；降压药；儿茶酚胺；尼卡地平；替米沙坦片；杜仲,"[-0.02898131, 0.047892302, -0.017043946, -0.00...",高血压 症 的 推荐 药 有些 什么 ？,0.315659
4,高血压无糖尿病的推荐药有些什么？,缬沙坦,"[-0.012565819, 0.008392007, -0.01837978, -0.02...",高血压 无 糖尿病 的 推荐 药 有些 什么 ？,0.396258
5,妊娠病的推荐药有些什么？,白芍；阿胶,"[-0.06891824, -0.02172745, 0.0016207923, 0.006...",妊娠 病 的 推荐 药 有些 什么 ？,0.413764
6,肾性高血压的推荐药有些什么？,士茯苓；缬沙坦联合硝苯地平；拜新；卡托普利；施慧达联合替米沙坦；科素亚；氯沙坦；替米沙坦；丙...,"[-0.013089769, 0.055687953, -0.022381965, -0.0...",肾性 高血压 的 推荐 药 有些 什么 ？,0.425781
7,中、高度原发性高血压的推荐药有些什么？,安博诺；拜新；硝苯地平,"[-0.0029709935, 0.03314647, -0.026325077, 0.01...",中 、 高度 原发性 高血压 的 推荐 药 有些 什么 ？,0.428327
8,原发性女性高血压病的推荐药有些什么？,卡托普利,"[-0.018945113, 0.06185117, -0.017091446, -0.00...",原发性 女性 高血压病 的 推荐 药 有些 什么 ？,0.446331
9,血瘀型高血压的推荐药有些什么？,氨氯地平；氨氯地平药物治疗；血府逐瘀汤加减；血府逐瘀汤；丹参注射液,"[-0.013250133, 0.028162444, -0.020013556, -0.0...",血瘀 型 高血压 的 推荐 药 有些 什么 ？,0.449114
